# WI-Projekt
Nr. 18

... <br>
... <br>
BA.WIN.22HS.VZa

... <br>
... <br>
BA.WIN.22HS.VZb

Pascal Kern <br>
kernpas1@students.zhaw.ch <br>
BA.WIN.22HS.VZa

## Setup

#### Get required Packages

In [1]:
import json
import os

from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

credentials = "credentials.json"

with open(credentials, 'r') as f:
    credentials = json.load(f)

# API-Schlüssel setzen
os.environ["OPENAI_API_KEY"] = credentials['openai']['api_key'] # OpenAI

#### Connect to DB

In [4]:
from langchain_community.utilities import SQLDatabase

#db = SQLDatabase.from_uri("sqlite:///DB\Chinook.db")
db = SQLDatabase.from_uri("sqlite:///DB\CarDealer.db")

print(db.dialect)
print(db.get_usable_table_names())

#db.run("SELECT * FROM Artist LIMIT 10;") # Chinook
db.run("SELECT * FROM cars LIMIT 10;") #Car dealer

sqlite
['cars', 'customers', 'mechanics', 'sales', 'salespersons', 'servicerecords']


"[(0, 'C7ZGWFLBUV', 'Toyota', 'Camry', 2019, 'A15 0LX', 21684.48, 'In Maintenance'), (1, 'C875HZ1AG6', 'Ford', 'Mustang', 2021, '435N', 48209.3, 'Sold'), (2, 'CDONEWSWFP', 'Honda', 'Civic', 2021, 'QBA-572', 25172.63, 'In Maintenance'), (3, 'C2UQZP803C', 'BMW', '320i', 2023, '5-92778W', 49999.41, 'In Maintenance'), (4, 'C2WFIK6WEP', 'Chevrolet', 'Malibu', 2020, '300 VLT', 36272.28, 'Sold'), (5, 'CGPCXD8911', 'Honda', 'Accord', 2023, 'TQJ 796', 46933.2, 'Sold'), (6, 'C4RQCO25I9', 'BMW', '320i', 2021, 'HPN 414', 24292.38, 'Available'), (7, 'C9LPTOS9KE', 'Mercedes', 'E-Class', 2021, '241 RAU', 49537.05, 'Available'), (8, 'C3U71M5BP5', 'Honda', 'Accord', 2020, '97-P320', 46892.88, 'Sold'), (9, 'CCKUBGF8P9', 'BMW', 'X5', 2023, 'JCC 664', 49575.53, 'In Maintenance')]"

### Setup Dynamic few-shot-prompt (vektorDB)

#### Load Examples

In [ ]:
# Laden der Beispiele aus der seperaten Datei
with open('Dynamic few-shot-prompts/few-shot-prompts.json', 'r') as f:
    examples = json.load(f)


#### Search based on semantic similarity

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# Auswählen passender Prompt-Beispiele
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

#### Create FewShotPromptTemplate

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

# Laden des template_prefix aus einer Textdatei
with open('Dynamic few-shot-prompts/template_prefix.txt', 'r') as f:
    system_prefix = f.read()

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix, 
    suffix="",
)


#### Create full prompt

In [ ]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [ ]:
# Preview
prompt_val = full_prompt.invoke(
    {
        "input": "Gib mir den Namen des Albums, mit den höchsten Gesamteinnahmen aus Track-Verkäufen, sowie die summierten Einnahmen dieser Tracks und die Anzahl verkaufter Tracks des Albums, basierend auf dem durchschnittlichen Einzelpreis der Tracks.",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

### Agent OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0) ## Which Large Language model are we using?

OpenAI_agent = create_sql_agent(
    llm, 
    toolkit= SQLDatabaseToolkit(llm=llm, db=db), 
    prompt=full_prompt,
    agent_type='tool-calling', 
    top_k=50,
    temperature=0,
    verbose=True) 


### Execute

In [ ]:
OpenAI_agent.invoke(
    {"input": "Zeige für jeden Kunden das Genre, das er am häufigsten gekauft hat, zusammen mit der Anzahl und dem Gesamtumsatz dieses Genres, und sortiere das Ergebnis nach dem Gesamtumsatz in absteigender Reihenfolge."}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Genre, Invoice, InvoiceLine, Track'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	LuÃ­s	GonÃ§alves	Embraer - Empresa Brasileira de AeronÃ¡utica S.A.	Av. Brigad

{'input': 'Zeige für jeden Kunden das Genre, das er am häufigsten gekauft hat, zusammen mit der Anzahl und dem Gesamtumsatz dieses Genres, und sortiere das Ergebnis nach dem Gesamtumsatz in absteigender Reihenfolge.',
 'output': 'Hier sind die Kunden mit dem Genre, das sie am häufigsten gekauft haben, zusammen mit der Anzahl der Käufe und dem Gesamtumsatz, sortiert nach dem Gesamtumsatz in absteigender Reihenfolge:\n\n1. Kunde 10: Genre "Rock", 29 Käufe, Gesamtumsatz 28,71\n2. Kunde 29: Genre "Rock", 25 Käufe, Gesamtumsatz 24,75\n3. Kunde 49: Genre "Rock", 22 Käufe, Gesamtumsatz 21,78\n4. Kunde 50: Genre "Rock", 22 Käufe, Gesamtumsatz 21,78\n5. Kunde 55: Genre "Rock", 22 Käufe, Gesamtumsatz 21,78\n6. Kunde 8: Genre "Rock", 21 Käufe, Gesamtumsatz 20,79\n7. Kunde 9: Genre "Rock", 21 Käufe, Gesamtumsatz 20,79\n8. Kunde 38: Genre "Rock", 21 Käufe, Gesamtumsatz 20,79\n9. Kunde 18: Genre "Rock", 19 Käufe, Gesamtumsatz 18,81\n10. Kunde 30: Genre "Rock", 19 Käufe, Gesamtumsatz 18,81\n\nDiese L